In [ ]:
! pip install -Uq tensorboard sentence-transformers datasets transformers

In [ ]:
import wandb
wandb.init(mode="disabled")

#### **Create and Prepare embedding dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("./amharic-dataset-binary", split="train")
dataset = dataset.filter(lambda row: row["label"]==1)#.select(range(100))
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'label'],
    num_rows: 31163
})

In [ ]:
# dataset = dataset.map(lambda row: {"query_in_passage": row["query"] in row["passage"]})
# dataset

In [ ]:
# sum(dataset["query_in_passage"]), len(dataset)

In [ ]:
# for i, b in enumerate(dataset["query_in_passage"][:100]):
#   if b:
#     print(i)

In [ ]:
dataset[32]

{'query_id': 'bc76b30a40c372418a9120f7dc69c012',
 'passage_id': 'bc76b30a40c372418a9120f7dc69c012',
 'query': 'ከኢራን ጋር የተደረገው ስምምነት መተግበር ለአለም ደህንነትና ለየኢራናውያ ንየኑሮ መሻሳል እንደሚጠቅም ጆን ኬሪ አስገነዘቡ',
 'passage': '\nጆን ኬሪ ከአሜሪካ ድምጽ ሬድዮ የፋርስ ባሀርሰላጤ አገልግሎት ጋር ባደረጉት ቃለ-መጠይቅ የዩናትድ ስቴትስ የምክር ቤት አባላት በኢራንና በአሜሪካ በተመራው የስድስት ሃገሮች ቡድን መካከል የተደረገውን ስምምነት ባያጸድቅ በመካከለኛው ምስራቅ ግጭትና ጦርነት እንዲባባስ የሚያደርግ መሆኑን እንደሚረዱ እተማመናለሁ ብልዋል።“ይህን ስምምነት ባናደርግ ኑሮ የኢራኑ አያቶላ እነዚህ ሰዎች ያስገኙት ነገር ስለሌለ ከነሱ ጋር የምንደራደርበት ምክንያት የለም ባሉ ነበር። የስራ ባልደረቦቻችን በኢራን ላይ ማዕቅቦች የተጣሉት ወደ ድርድርና ስምምነት እንድታመራ ለማድረግ ነው እያሉ ማዕቀቡ እንዲቀጥል የሚደግፉበት ምክንያት ምንድነው? አሁን የተገኘውን ስምምነት ባይቀባሉ በመካከለኛው ምስራቅ ላይ ትልቅ አደጋ ይደቀናል” በማለት ጆን ኬሪ አስገንዝበዋል። የዩናትድ ስቴስ አስተዳደር ኢራን የታሰሩት አሜሪካውያን እንዲፈቱ መጠየቅን የተካተቱባቸው አጀንዳዎችን ጨምሮ መግፋት ነበረበት የሚሉ ነቀፊታዎች ሲቀርቡ ቆይተዋል። ጆን ኬሪ ታድያ ሌሎች ነጥቦችንም ጨምረን ቢሆን ኖሮ የትም አንደርስም ነበር ብለዋል።“ከዚህ በፊትም እንደገለጽነው በኢራንና በአለም ማህበረሰቡ ወይም በዩናይትድ ስቴትስ እና በኢራን መካከል በሚደረገው ድርድር ሌሎች ነጥቦችንም አክለን ቢሆን ኖሮ ለ 10 አመታት ያህል ብንደራደርም እንኳን የትም አንደርስም ነበር። ነጥቡ የኑክሌር ጉዳይ በመሆኑ ከተደረሰው መሻሻል

In [ ]:
# rename columns
dataset = dataset.rename_column("query", "anchor")
dataset = dataset.rename_column("passage", "positive")

In [ ]:
# Add an id column to the dataset
dataset = dataset.add_column("id", range(len(dataset)))
dataset

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
    num_rows: 31163
})

In [ ]:
# Split dataset into a 10% test set
dataset = dataset.train_test_split(test_size=0.1, seed=7)
dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
        num_rows: 28046
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
        num_rows: 3117
    })
})

In [ ]:
# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Creating json from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

12602505

#### **Create baseline and evaluate pretrained model**

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets

# load test dataset
test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

corpus_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
    num_rows: 31163
})

In [ ]:
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
) # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
) # Our queries (qid => question)

In [ ]:
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}
for q_id in queries:
  relevant_docs[q_id] = [q_id]

#### **Evaluate Amharic Embedding model**

List of models to evaluate:
- intfloat/multilingual-e5-large-instruct
- Alibaba-NLP/gte-modernbert-base
- Alibaba-NLP/gte-multilingual-base


In [ ]:
# BGE Small
import torch
from sentence_transformers import SentenceTransformer

# model_id = "rasyosef/roberta-amharic-text-embedding-base"
# model_id = "rasyosef/roberta-amharic-text-embedding-medium"
# model_id = "rasyosef/bert-amharic-text-embedding-medium"

# # OTHER MODELS
# model_id = "intfloat/multilingual-e5-large-instruct"
# model_id = "Alibaba-NLP/gte-modernbert-base"
# model_id = "Alibaba-NLP/gte-multilingual-base"
model_id = "Snowflake/snowflake-arctic-embed-l-v2.0"

# Load a model
model = SentenceTransformer(
    model_id,
    device="cuda" if torch.cuda.is_available() else "cpu",
    trust_remote_code=True
)

In [ ]:
EMBED_DIM = model.get_sentence_embedding_dimension()
matryoshka_dimensions = [EMBED_DIM]

print("Embedding Dimension:", EMBED_DIM)

matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
  ir_evaluator = InformationRetrievalEvaluator(
      queries=queries,
      corpus=corpus,
      relevant_docs=relevant_docs,
      batch_size=64,
      name=f"dim_{dim}",
      truncate_dim=dim,
      score_functions={"cosine": cos_sim},
      mrr_at_k=[10, 100],
      ndcg_at_k=[10, 100],
      precision_recall_at_k=[5, 10, 50, 100],
      corpus_chunk_size=512,
      show_progress_bar=True
  )
  matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

Embedding Dimension: 1024


In [ ]:
# Evaluate the model
results = evaluator(model)

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 61/61 [07:54<00:00,  7.78s/it]


In [ ]:
# Snowflake/snowflake-arctic-embed-l-v2.0

dim = EMBED_DIM
metrics = [
    f"dim_{dim}_cosine_recall@10",
    f"dim_{dim}_cosine_recall@50",
    f"dim_{dim}_cosine_recall@100",
    f"dim_{dim}_cosine_mrr@10",
    f"dim_{dim}_cosine_mrr@100",
    f"dim_{dim}_cosine_ndcg@10",
    f"dim_{dim}_cosine_ndcg@100"
  ]

for key in metrics:
  metric_name = key.split("_")[-1]
  print(f"{metric_name}: {results[key]}")

recall@10: 0.8675008020532563
recall@50: 0.9406480590311197
recall@100: 0.9570099454603785
mrr@10: 0.7192691385184166
mrr@100: 0.7232914016418369
ndcg@10: 0.7554149227417887
ndcg@100: 0.7747152074463952


### **Example**

In [ ]:
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "የዋጋ ግሽበት በባለሙያዎች እና ነዋሪዎች አተያይ",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡",
  "ለኢትዮጵያ ብሔራዊ ባንክ ዋጋን የማረጋጋት ቀዳሚ ዓላማ ጋር የተጣጣሙ የገንዘብ ፖሊሲ ምክረ ሀሳቦችን እንዲሰጥ የተቋቋመው የኢትዮጵያ ብሔራዊ ባንክ የገንዘብ ፖሊሲ ኮሚቴ እስካለፈው ህዳር ወር የነበረው እአአ የ2024 የዋጋ ግሽበት በተለይምምግብ ነክ ምርቶች ላይ ከአንድ ዓመት በፊት ከነበው ጋር ሲነጻጸር መረጋጋት ማሳየቱን ጠቁሟል፡፡ ዶይቼ ቬለ ያነጋገራቸው የአዲስ አበባ ነዋሪዎች ግን በዚህ የሚስማሙ አይመስልም፡፡ ከአምና አንጻር ያልጨመረ ነገር የለም ባይ ናቸው፡፡ የኢኮኖሚ  ባለሙያም በሰጡን አስተያየት ጭማሪው በሁሉም ረገድ የተስተዋለ በመሆኑ የመንግስት ወጪን በመቀነስ ግብርናው ላይ አተኩሮ መስራት ምናልባትም የዋጋ መረጋጋቱን ሊያመጣ ይችላል ይላሉ፡፡"
]
embeddings = model.encode(sentences)

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.3620, 0.6287, 0.2046],
        [0.3620, 1.0000, 0.2787, 0.4756],
        [0.6287, 0.2787, 1.0000, 0.2198],
        [0.2046, 0.4756, 0.2198, 1.0000]])
